<a href="https://colab.research.google.com/github/kazirafi71/Thesis/blob/master/New_Resume_Dataset.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [66]:
pip install tensorflow-text


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [67]:

import tensorflow as tf
import tensorflow_hub as hub



In [68]:
pip install --upgrade tensorflow-hub

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [69]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [70]:
!pip install np_utils

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [71]:
import numpy as np
import pandas as pd
from plotly.offline import iplot
import matplotlib.pyplot as plt
import seaborn as sns
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding, LSTM, SpatialDropout1D
from sklearn.model_selection import train_test_split
from keras.utils.np_utils import to_categorical
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.layers import Dropout
import re
from nltk.corpus import stopwords
from nltk import word_tokenize
STOPWORDS = set(stopwords.words('english'))

In [72]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [74]:
import pandas as pd

df = pd.read_csv("/content/drive/MyDrive/CV Sorting Project/resume_files.csv")
df.head(100)

,Unnamed: 0,Label,skills,experience
0,0,ACCOUNTANT,"['Partnership', 'Compliance', 'Protocols', 'In...","['Services', 'Financial statements, Bank recon..."
1,1,ACCOUNTANT,"['Process', 'Certification', 'Vendors', 'Tax',...","['Accountant Jul 2014 to Current', 'Company Na..."
2,2,ACCOUNTANT,"['Contracts', 'Inventory', 'Compliance', 'Peop...","['08/2014 to Current', 'Accountant Company Nam..."
3,3,ACCOUNTANT,"['Compliance', 'Invoices', 'Analytical', 'Regu...",['Company Name August 2012 to December 2014 Ac...
4,4,ACCOUNTANT,"['Illustrator', 'Contracts', 'Inventory', 'Inv...","['October 2011', 'to', 'August 2015', 'Company..."
...,...,...,...,...
95,98,ACCOUNTANT,"['Statistical analysis', 'Contracts', 'Invento...",['Company Name August 2013 to Current Senior A...
96,99,ACCOUNTANT,"['Microsoft office suite', 'Partnership', 'Adm...","['11/2012 to 09/2013', 'Volunteer Accountant C..."
97,101,ACCOUNTANT,"['Real estate', 'Compliance', 'Peoplesoft', 'R...",[]
98,103,ACCOUNTANT,"['Inventory', 'Compliance', 'Peoplesoft', 'Inv...","['Payroll Accountant Apr 2014 to Current', 'Co..."


In [75]:
df.groupby('Label').describe()

Unnamed: 0                                          \
                            count       mean        std  min    25%   50%   
Label                                                                       
ACCOUNTANT                  112.0  57.794643  34.074148  0.0  28.75  57.5   
ADVOCATE                    110.0  58.081818  33.630789  0.0  29.25  58.5   
AGRICULTURE                  63.0  31.000000  18.330303  0.0  15.50  31.0   
APPAREL                      93.0  48.827957  27.993254  0.0  25.00  49.0   
ARTS                         99.0  51.444444  30.064971  0.0  26.50  51.0   
AUTOMOBILE                   35.0  17.885714  10.428387  0.0   9.50  18.0   
AVIATION                    115.0  57.295652  33.730088  0.0  28.50  57.0   
BANKING                     112.0  56.875000  33.035914  0.0  28.75  56.5   
BPO                          22.0  10.500000   6.493587  0.0   5.25  10.5   
BUSINESS-DEVELOPMENT        115.0  59.634783  34.893404  0.0  30.50  60.0   
CHEF                        114.0  58.868421  34.674597  0.0  29.25  60.5   
CONSTRUCTION                105.0  55.085714  32.199532  0.0  28.00  56.0   
CONSULTANT                  111.0  57.702703  33.279968  0.0  29.50  58.0   
DESIGNER                     99.0  53.232323  31.289246  0.0  26.50  54.0   
DIGITAL-MEDIA                92.0  48.652174  27.812336  0.0  25.75  49.5   
ENGINEERING                 115.0  58.895652  34.163275  0.0  29.50  59.0   
FINANCE                     113.0  58.318584  33.536032  0.0  30.00  58.0   
FITNESS                     113.0  58.318584  34.341839  0.0  28.00  59.0   
HEALTHCARE                  110.0  56.327273  33.072650  0.0  28.25  56.5   
HR                          109.0  54.935780  31.714956  0.0  28.00  55.0   
INFORMATION-TECHNOLOGY      118.0  59.440678  34.624615  0.0  30.25  59.5   
PUBLIC-RELATIONS            104.0  55.625000  32.506067  0.0  27.50  56.5   
SALES                       115.0  57.304348  33.711150  0.0  28.50  57.0   
TEACHER                      92.0  50.391304  28.803600  1.0  26.75  49.5   

                                      
                          75%    max  
Label                                 
ACCOUNTANT              86.25  117.0  
ADVOCATE                86.75  117.0  
AGRICULTURE             46.50   62.0  
APPAREL                 73.00   96.0  
ARTS                    77.50  102.0  
AUTOMOBILE              26.50   35.0  
AVIATION                86.50  116.0  
BANKING                 85.25  114.0  
BPO                     15.75   21.0  
BUSINESS-DEVELOPMENT    89.50  119.0  
CHEF                    88.75  117.0  
CONSTRUCTION            82.00  111.0  
CONSULTANT              86.50  114.0  
DESIGNER                80.50  106.0  
DIGITAL-MEDIA           72.25   95.0  
ENGINEERING             88.50  117.0  
FINANCE                 86.00  116.0  
FITNESS                 88.00  116.0  
HEALTHCARE              83.75  114.0  
HR                      82.00  109.0  
INFORMATION-TECHNOLOGY  88.75  119.0  
PUBLIC-RELATIONS        83.25  110.0  
SALES                   86.50  115.0  
TEACHER                 73.25  101.0

In [76]:
df['Label'].value_counts()

INFORMATION-TECHNOLOGY    118
SALES                     115
AVIATION                  115
ENGINEERING               115
BUSINESS-DEVELOPMENT      115
CHEF                      114
FITNESS                   113
FINANCE                   113
ACCOUNTANT                112
BANKING                   112
CONSULTANT                111
ADVOCATE                  110
HEALTHCARE                110
HR                        109
CONSTRUCTION              105
PUBLIC-RELATIONS          104
DESIGNER                   99
ARTS                       99
APPAREL                    93
DIGITAL-MEDIA              92
TEACHER                    92
AGRICULTURE                63
AUTOMOBILE                 35
BPO                        22
Name: Label, dtype: int64

In [77]:
def example_complain(index):
    example = df[df.index == index][['skills', 'Label']].values[0]
    if len(example) > 0:
        print(example[0])
        print()
        print('Category:', example[1])

In [78]:
example_complain(10)

['Ms excel', 'Partnership', 'Inventory', 'Contracts', 'Training', 'Windows', 'Accounts receivable', 'Audit', 'Technical', 'Transactions', 'Staffing', 'Purchase orders', 'Merchandising', 'Public relations', 'Microsoft office', 'Quality assurance', 'Safety', 'Accounts payable', 'Payroll', 'Sales', 'Supervisor', 'Writing', 'Reports', 'Forecasting', 'Budgeting', 'Customer service', 'Logistics', 'Warehouse', 'Economics', 'Reporting', 'Reconcile', 'Schedule', 'Retail', 'Networking', 'Accounting', 'Ordering', 'Excel', 'Process', 'System', 'Vendors', 'Standard operating procedures', 'Tax', 'Troubleshooting', 'Project management', 'Billing', 'Operations', 'Assembly']

Category: ACCOUNTANT


In [79]:
data = df.reset_index(drop=True)
REPLACE_BY_SPACE_RE = re.compile('[/(){}\[\]\|@,;]')
BAD_SYMBOLS_RE = re.compile('[^0-9a-z #+_]')
STOPWORDS = set(stopwords.words('english'))

def clean_text(text):
    """
        text: a string
        
        return: modified initial string
    """
    text = text.lower() # lowercase text
    text = REPLACE_BY_SPACE_RE.sub(' ', text) # replace REPLACE_BY_SPACE_RE symbols by space in text. substitute the matched string in REPLACE_BY_SPACE_RE with space.
    text = BAD_SYMBOLS_RE.sub('', text) # remove symbols which are in BAD_SYMBOLS_RE from text. substitute the matched string in BAD_SYMBOLS_RE with nothing. 
    text = text.replace('x', '')
#    text = re.sub(r'\W+', '', text)
    text = ' '.join(word for word in text.split() if word not in STOPWORDS) # remove stopwors from text
    return text
df['skills'] = data['skills'].apply(clean_text)

In [80]:
example_complain(10)

ms ecel partnership inventory contracts training windows accounts receivable audit technical transactions staffing purchase orders merchandising public relations microsoft office quality assurance safety accounts payable payroll sales supervisor writing reports forecasting budgeting customer service logistics warehouse economics reporting reconcile schedule retail networking accounting ordering ecel process system vendors standard operating procedures ta troubleshooting project management billing operations assembly

Category: ACCOUNTANT


In [81]:
# The maximum number of words to be used. (most frequent)
MAX_NB_WORDS = 50000
# Max number of words in each complaint.
MAX_SEQUENCE_LENGTH = 250
# This is fixed.
EMBEDDING_DIM = 100

tokenizer = Tokenizer(num_words=MAX_NB_WORDS, filters='!"#$%&()*+,-./:;<=>?@[\]^_`{|}~', lower=True)
tokenizer.fit_on_texts(df['skills'].values)
word_index = tokenizer.word_index
print('Found %s unique tokens.' % len(word_index))

Found 638 unique tokens.


In [82]:
X = tokenizer.texts_to_sequences(df['skills'].values)
X = pad_sequences(X, maxlen=MAX_SEQUENCE_LENGTH)
print('Shape of df tensor:', X.shape)

Shape of df tensor: (2386, 250)


In [83]:
Y = pd.get_dummies(df['Label']).values
print('Shape of label tensor:', Y.shape)

Shape of label tensor: (2386, 24)


In [84]:
X_train, X_test, Y_train, Y_test = train_test_split(X,Y, test_size = 0.10, random_state = 42)
print(X_train.shape,Y_train.shape)
print(X_test.shape,Y_test.shape)

(2147, 250) (2147, 24)
(239, 250) (239, 24)


In [90]:
model = Sequential()
model.add(Embedding(MAX_NB_WORDS, EMBEDDING_DIM, input_length=X.shape[1]))
model.add(SpatialDropout1D(0.2))
model.add(LSTM(100, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(24, activation='sigmoid'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
print(model.summary())

Model: "sequential_15"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_15 (Embedding)    (None, 250, 100)          5000000   
                                                                 
 spatial_dropout1d_9 (Spatia  (None, 250, 100)         0         
 lDropout1D)                                                     
                                                                 
 lstm_15 (LSTM)              (None, 100)               80400     
                                                                 
 dense_15 (Dense)            (None, 24)                2424      
                                                                 
Total params: 5,082,824
Trainable params: 5,082,824
Non-trainable params: 0
_________________________________________________________________
None


In [91]:
epochs = 30
batch_size = 64

history = model.fit(X_train, Y_train, epochs=epochs,
                    batch_size=batch_size,validation_split=0.1,
                    callbacks=[EarlyStopping(monitor='val_loss', patience=5,min_delta=0.0001, verbose=0, mode='auto')])


Epoch 1/30


Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: closure mismatch, requested ('self', 'step_function'), but source function had ()
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: closure mismatch, requested ('self', 'step_function'), but source function had ()
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
31/31 [==============================] - ETA: 0s - loss: 3.1514 - accuracy: 0.0932

Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: closure mismatch, requested ('self', 'step_function'), but source function had ()
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: closure mismatch, requested ('self', 'step_function'), but source function had ()
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
31/31 [==============================] - 29s 851ms/step - loss: 3.1514 - accuracy: 0.0932 - val_loss: 3.0851 - val_accuracy: 0.1628
Epoch 2/30
31/31 [==============================] - 23s 736ms/step - loss: 2.9840 - accuracy: 0.1284 - val_loss: 2.8610 - val_accuracy: 0.0930
Epoch 3/30
31/31 [==============================] - 23s 735ms/step - loss: 2.6956 - accuracy: 0.2008 - val_loss: 2.6471 - val_accuracy: 0.1953
Epoch 4/30
31/31 [==============================] - 23s 735ms/step - loss: 2.5138 - accuracy: 0.2598 - val_loss: 2.5525 - val_accuracy: 0.2279
Epoch 5/30
31/31 [==============================] - 22s 725ms/step - loss: 2.3437 - accuracy: 0.